Ref: 
1. https://github.com/langchain-ai/langchain/blob/master/cookbook/rag_fusion.ipynb?ref=blog.langchain.dev
2. https://github.com/Raudaschl/rag-fusion

RAG-Fusion, a search methodology that aims to bridge the gap between traditional search paradigms and the multifaceted dimensions of human queries.  

Inspired by the capabilities of Retrieval Augmented Generation (RAG), this project goes a step further by employing `multiple query generation` and `Reciprocal Rank Fusion` to re-rank search results.

In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)

import rich

In [2]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

local_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/Users/sean_liu/miniconda3/envs/langchain/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/sean_liu/miniconda3/envs/langchain/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
all_documents = {
    "doc1": "Climate change and economic impact.",
    "doc2": "Public health concerns due to climate change.",
    "doc3": "Climate change: A social perspective.",
    "doc4": "Technological solutions to climate change.",
    "doc5": "Policy changes needed to combat climate change.",
    "doc6": "Climate change and its impact on biodiversity.",
    "doc7": "Climate change: The science and models.",
    "doc8": "Global warming: A subset of climate change.",
    "doc9": "How climate change affects daily weather.",
    "doc10": "The history of climate change activism.",
}

#### Using Chroma

In [5]:
from langchain_community.vectorstores import Chroma
# vectorstore = Chroma.from_texts(all_documents.values(), embedding=OpenAIEmbeddings(), collection_name="rag_fusion", persist_directory="./rag_fusion_db")
vectorstore = Chroma.from_texts(all_documents.values(), embedding=local_embeddings, collection_name="rag_fusion", persist_directory="./rag_fusion_db")

#### Using FAISS

In [20]:
# from langchain.vectorstores.faiss import FAISS
# vectorstore = FAISS.from_texts(all_documents.values(), embedding=OpenAIEmbeddings())

## Define the Query Generator

In [6]:
from langchain import hub

prompt = hub.pull("langchain-ai/rag-fusion-query-generation")
rich.print(prompt)

ChatPromptTemplate(
    input_variables=['original_query'],
    input_types={},
    partial_variables={},
    metadata={
        'lc_hub_owner': 'langchain-ai',
        'lc_hub_repo': 'rag-fusion-query-generation',
        'lc_hub_commit_hash': '478b448e096b977446865108fad34282e6e1a84ae8b8540572ed0df238229a11'
    },
    messages=[
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=[],
                input_types={},
                partial_variables={},
                template='You are a helpful assistant that generates multiple search queries based on a single 
input query.'
            ),
            additional_kwargs={}
        ),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['original_query'],
                input_types={},
                partial_variables={},
                template='Generate multiple search queries related to: {original_query}'
            ),
            additional_kwargs={}
        ),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=[],
                input_types={},
                partial_variables={},
                template='OUTPUT (4 queries):'
            ),
            additional_kwargs={}
        )
    ]
)

In [7]:
# def split_lines(x):
#     return x.split("\n")

# generate_queries = prompt | ChatOpenAI(model="gpt-4o-mini", temperature=0.5) | StrOutputParser() | split_lines

generate_queries = prompt | ChatOpenAI(model="gpt-4o-mini", temperature=0.5) | StrOutputParser() | (lambda x: x.split("\n"))

## Define the full chain
We can now put it all together and define the full chain.
1. Generate a bunch queries
2. Retrieval by each query
3. Joins all the results and ordered by Reciprocal Rank Fusion

In [8]:
retriever = vectorstore.as_retriever()

In [9]:
from langchain.load import dumps, loads

def rrf(results: list[list], k=60):
    fused_scores = {}
    for docs in results:
        # assumes the docs are returned in the order of relevance
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            fused_scores[doc_str] += 1/(rank+k)

    reranked_results = [
        (loads(doc_str), score) for doc_str, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]
    
    return reranked_results

In [10]:
chain = generate_queries | retriever.map() | rrf

In [11]:
original_queries = "Impact of climat change"
final_results = chain.invoke({"original_query": original_queries})

In [12]:
rich.print(final_results)

[
    (Document(metadata={}, page_content='Climate change and economic impact.'), 0.06585580821434867),
    (Document(metadata={}, page_content='Climate change and its impact on biodiversity.'), 0.03306010928961749),
    (Document(metadata={}, page_content='Public health concerns due to climate change.'), 0.032539682539682535),
    (Document(metadata={}, page_content='How climate change affects daily weather.'), 0.03200204813108039),
    (Document(metadata={}, page_content='Policy changes needed to combat climate change.'), 0.03200204813108039),
    (Document(metadata={}, page_content='Climate change: A social perspective.'), 0.01639344262295082),
    (Document(metadata={}, page_content='Climate change: The science and models.'), 0.01639344262295082),
    (Document(metadata={}, page_content='Technological solutions to climate change.'), 0.016129032258064516),
    (Document(metadata={}, page_content='Global warming: A subset of climate change.'), 0.015873015873015872)
]